In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
#load no dataset
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, test_size=0.3)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.3)

In [3]:
#Scalando as variáveis numéricas
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [13]:
#Montando o modelo - utilizando apenas 1 output e sem função de ativação visto q queremos um valor. A loss é MSE
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=30, activation='relu', input_shape=X_train.shape[1:]))
model.add(keras.layers.Dense(units=1))

In [14]:
#compilando o modelo
model.compile(loss='mean_squared_error', optimizer='sgd')

In [15]:
#fitando o modelo
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
317/317 [==============================] - 1s 4ms/step - loss: 0.8363 - val_loss: 0.5431
Epoch 2/20
317/317 [==============================] - 1s 3ms/step - loss: 0.5259 - val_loss: 0.4912
Epoch 3/20
317/317 [==============================] - 1s 2ms/step - loss: 0.5067 - val_loss: 0.4961
Epoch 4/20
317/317 [==============================] - 1s 2ms/step - loss: 0.6298 - val_loss: 0.5005
Epoch 5/20
317/317 [==============================] - 1s 2ms/step - loss: 1.0389 - val_loss: 0.4804
Epoch 6/20
317/317 [==============================] - 1s 2ms/step - loss: 0.4607 - val_loss: 0.5612
Epoch 7/20
317/317 [==============================] - 1s 2ms/step - loss: 0.4394 - val_loss: 0.4308
Epoch 8/20
317/317 [==============================] - 1s 2ms/step - loss: 0.4229 - val_loss: 0.4104
Epoch 9/20
317/317 [==============================] - 1s 2ms/step - loss: 0.4142 - val_loss: 0.4131
Epoch 10/20
317/317 [==============================] - 1s 2ms/step - loss: 0.4078 - val_loss: 0.4032

In [16]:
#testando no hold-out
mse_test = model.evaluate(X_test, y_test)

194/194 [==============================] - 0s 1ms/step - loss: 0.3746


In [19]:
#prevendo novos casos:
X_new = X_test[:3]
predict = model.predict(X_new)
predict

array([[2.405637 ],
       [2.6174054],
       [2.4074845]], dtype=float32)

# Wide & Deep Neural Network Architecture - Functional API

In [21]:
#Construindo um modelo wide & deep que liga os inputs ao ultimo layer antes do output e concatena para o output.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_],outputs=[output])

In [23]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [24]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
317/317 [==============================] - 1s 2ms/step - loss: 1.6488 - val_loss: 0.6045
Epoch 2/30
317/317 [==============================] - 0s 1ms/step - loss: 0.5377 - val_loss: 0.4641
Epoch 3/30
317/317 [==============================] - 0s 1ms/step - loss: 0.4499 - val_loss: 0.4171
Epoch 4/30
317/317 [==============================] - 0s 1ms/step - loss: 0.4357 - val_loss: 0.3993
Epoch 5/30
317/317 [==============================] - 0s 1ms/step - loss: 0.4065 - val_loss: 0.3937
Epoch 6/30
317/317 [==============================] - 0s 1ms/step - loss: 0.4116 - val_loss: 0.3763
Epoch 7/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3890 - val_loss: 0.3742
Epoch 8/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3809 - val_loss: 0.3627
Epoch 9/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3703 - val_loss: 0.3658
Epoch 10/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3704 - val_loss: 0.3609

In [25]:
mse_test = model.evaluate(X_test, y_test)

194/194 [==============================] - 0s 650us/step - loss: 0.3068


# Save and Load model

In [26]:
#salvando o modelo
model.save('wide_and_deep_model.h5')

In [27]:
#dando load em modelo
model_l = keras.models.load_model('wide_and_deep_model.h5')

# Callbacks para salvar treino (save_best_only = True para salvar melhor fit)

In [31]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('wide_and_deep_model.h5', save_best_only=True)

In [32]:
history = model_l.fit(X_train, 
                      y_train, 
                      epochs=30, 
                      validation_data=(X_valid, y_valid),
                      callbacks=[checkpoint_cb])

Epoch 1/30
317/317 [==============================] - 1s 2ms/step - loss: 0.3080 - val_loss: 0.3091
Epoch 2/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3085 - val_loss: 0.3077
Epoch 3/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3139 - val_loss: 0.3006
Epoch 4/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2982 - val_loss: 0.3006
Epoch 5/30
317/317 [==============================] - 0s 1ms/step - loss: 0.3009 - val_loss: 0.3101
Epoch 6/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2938 - val_loss: 0.3117
Epoch 7/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2973 - val_loss: 0.2980
Epoch 8/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2927 - val_loss: 0.3167
Epoch 9/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2937 - val_loss: 0.3035
Epoch 10/30
317/317 [==============================] - 0s 1ms/step - loss: 0.2989 - val_loss: 0.3040